In [1]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from sklearn.manifold import LocallyLinearEmbedding as LLE
from sklearn.preprocessing import StandardScaler
import os
import matplotlib.image as image
from matplotlib import pyplot
import tensorflow as tf
from PIL import Image

In [2]:
p = os.path.abspath('.')
t = p + '/Pictures/Reconstruction/Thermal/'
v = p + '/Pictures/Reconstruction/visual/'

ti = []
vi = []

# Number of channels
n = 3

# The height and width of resized thermal spectrum image
H = 56*1
W = 64*1

# The height an width of resized visible spectrum image
H1 = 56*1
W1 = 64*1

for i in range(5,26):
    
    
#   using tensorflow to change image to numpy array (t_i - ith image in thermal, v_i - ith image in thermal)  
    t_i = tf.io.read_file(t + '%d.png' % (i))
    t_i = tf.image.decode_png(t_i, channels = n)
    t_i = tf.image.resize(t_i,(H,W)).numpy()
    t_i = t_i.reshape(1,H,W,n)
    ti.append(t_i)
    
    v_i = tf.io.read_file(v + '%d.jpg' % (i))
    v_i = tf.image.decode_png(v_i, channels = n)
    v_i = tf.image.resize(v_i,(H1,W1)).numpy()
    v_i = t_i.reshape(1,H1,W1,n)
    vi.append(v_i)

    
    
X_img = np.concatenate(ti, axis = 0)
Y_img = np.concatenate(vi, axis = 0)

In [3]:
X_img = X_img[:-1] # training thermal 
X_q = X_img[-1][np.newaxis,:] # testing thermal 
Y_img = Y_img[:-1] # training visible 
Y_q = Y_img[-1][np.newaxis,:] # testing visible
X_q.shape

(1, 56, 64, 3)

In [4]:
# function to centralise input and also return the mean
def Central(X_flat): 
    mu_x = X_flat
    mu_x = np.mean(mu_x, axis = 0)
    mu_x = (mu_x[np.newaxis,:])
    Xc = X_flat - mu_x
    return Xc,mu_x
    

def Global_Training(X_img,Y_img):
#   N - No. of images, H - height of image, W - width of image, C - No. of Channel 
#   X_img and Y_img are of shapes (N,H,W,C) for thermal and visible spectrum.
    N,H,W,C1 = X_img.shape
    N,H,W,C2 = Y_img.shape
        
    Xm,u_x = Central(X_img.reshape(N,-1))
    Ym,u_y = Central(Y_img.reshape(N,-1))
    
    pcax = PCA()
    pcay = PCA()  

#   the number of eigen vectore to be chosen can <= d or H*C*W in this case 
    pcax.fit(Xm)
    pcay.fit(Ym)
        
    X_eig = pcax.transform(Xm)
    Y_eig = pcay.transform(Ym)  
        
    Xm_eig,v_x = Central(X_eig)   
    Ym_eig,v_y = Central(Y_eig)
      
    cca = CCA()
    cca.fit(Xm_eig,Ym_eig)  
         
    X_coh, Y_coh = cca.transform(Xm_eig,Ym_eig) 
        
    return pcax, pcay, cca, u_x, u_y, v_x, v_y, X_coh, Y_coh

def Global_Recont(pcax, pcay, cca, u_x, u_y, v_x, v_y, X_coh, Y_coh, X_sam, Y_sam, K):
    
    N,H,W,C2 = Y_sam.shape
    
    Xs_eig = pcax.transform(X_sam.reshape(X_sam.shape[0],-1)-u_x)
    Xs_coh = cca.transform(Xs_eig-v_x)
    
    idx = np.argsort(np.sum(np.square(X_coh-Xs_coh), axis = 1))[:K]
    G = np.zeros((K,K))
    Wi = np.zeros((1,K))
    Ys_coh = np.zeros_like(Xs_coh)
    
    for i in range(K):
        for j in range(K):
            G[i,j] = np.sum( (Xs_coh[0] - X_coh[idx[i]])*(Xs_coh[0] - X_coh[idx[j]]) )
        
    
    G_inv = np.linalg.pinv(G)
    
    for i in range(K):
        Wi[0,i] = (np.sum(G_inv[:,i])/(np.sum(G_inv)))
        Ys_coh += (Wi[0,i]*Y_coh[idx[i]])
    
    Ys_eig = cca.inverse_transform(Ys_coh) + v_y
    Y_rec = pcay.inverse_transform(Ys_eig) + u_y
     
    return Y_rec.reshape(Y_sam.shape)

def LocalRef_Training(Y_rec,Y_img,X_img,P):
    
    N,H,W,C1 = X_img.shape
    N,H,W,C2 = Y_img.shape
    
    Hp = []
    
    for i in range(int(H-P)):
        for j in range(int(W-P)):
            cca = CCA(max_iter=5000)
            
            H_x = (Y_img[:,i:P+i,j:P+j,:] - Y_rec[:,i:P+i,j:P+j,:]).reshape(N,-1)
            Hm_x,l_x = Central(H_x)
            
            H_y = (X_img[:,i:P+i,j:P+j,:] - Y_rec[:,i:P+i,j:P+j,:]).reshape(N,-1)
            Hm_y,l_y = Central(H_y)
            
            cca.fit(Hm_x,Hm_y)
            
            Hm_x, Hm_y = cca.transform(Hm_x,Hm_y) 
            Hp.append([Hm_x,l_x,Hm_y,l_y,cca])
            
    
    return Hp

def LocalRef_Recont(Hp, X_sam, Y_rec, K, P):
    
    H_x = X_sam-Y_rec
    N,H,W,C = H_x.shape
    k = 0
    Y_sam = np.zeros_like(X_sam)

#  The patches are retrieved by a 9*9 sliding window that moves through an image and created 
#  with overlapping pixel information   

    for i in range(int(H-P)):
        for j in range(int(W-P)):    
            
            Hm_x,l_x,Hm_y,l_y,cca = Hp[k]
            k += 1
            
            Hcoh_x = cca.transform(H_x[:,i:P+i,j:P+j,:].reshape(N,-1)- l_x)
            
            idx = np.argsort(np.sum(np.square(Hcoh_x-Hm_x), axis = 1))[:K]
            G = np.zeros((K,K))
            Wi = np.zeros((1,K))
            Hcoh_y = np.zeros_like(Hcoh_x)
            
            for i1 in range(K):
                for j1 in range(K):
                    G[i1,j1] = np.sum( (Hcoh_x[0] - Hm_x[idx[i1]])*(Hcoh_x[0] - Hm_x[idx[j1]]) )
                            
                    
            G_inv = np.linalg.pinv(G)
            
            for i2 in range(K):
                Wi[0,i2] = np.sum(G_inv[:,i2])/(np.sum(G_inv))
                Hcoh_y += Wi[0,i2]*Hm_y[idx[i2]]
            
            H_y = cca.inverse_transform(Hcoh_y) + l_y
               
            Y_sam[:,i:P+i,j:P+j,:] = H_y.reshape(Y_rec[:,i:P+i,j:P+j,:].shape)  + Y_rec[:,i:P+i,j:P+j,:]
            
    return Y_sam 

In [5]:
P = 9 #size of patch 9*9 
K = 5 #number of neighbours for Locally linear Embeddings

# Global training and reconstruction 
p_cax, pcay, cca, u_x, u_y, v_x, v_y, X_coh, Y_coh = Global_Training(X_img,Y_img)
a = Global_Recont(p_cax, pcay, cca, u_x, u_y, v_x, v_y, X_coh, Y_coh, X_q, Y_q, K)

# Local Refinement training and reconstruction 
Y_rec = Global_Recont(p_cax, pcay, cca, u_x, u_y, v_x, v_y, X_coh, Y_coh, X_img, Y_img, K)    
Hp = LocalRef_Training(Y_rec,Y_img,X_img,P)
Y_sam = LocalRef_Recont(Hp, X_q, a, K, P)

In [7]:
N,H,W,C = Y_sam.shape
data = (Y_sam.reshape(H,W,C)).astype(np.uint8)
img = Image.fromarray(data, 'RGB')
img.show()